# Aplicar preprocesado y formatear texto
En este notebook vamos a aplicar el preprocesado que hemos estudiado en el notebook anterior al corpus que tenemos. Tras ello, vamos a pasar el texto a formato conllu, que es lo que el modelo que usamos para resolver las referencias usa como entrada.

# 1 Aplicar preprocesado

## 1.1 Imports
Vamos a poner todos los imports que necesitamos para aplicar el preprocesado.

In [ ]:
import stanza
import re
import json

In [ ]:
stanza.download('es')

## 1.2 Funciones auxiliares
Ahora vamos a definir nuestra función para preprocesar el texto y todas sus sub-funciones. Recordemos que primero vamos a pasarle nuestro tokenizador personalizado y despues el modelo stanza para sacar las part-of-speech-tags.

In [ ]:
IMPERATIVE_MAP = {
    r"(?:C|c)óme((?:me|te|se|nos|os)?)((?:lo|la|le|los|las|les)?)": {
        "base": "come",
        "lemma": "comer",
        "upos": "VERB"
    },
    r"(?:C|c)om[eé]d((?:me|te|se|nos|os)?)((?:lo|la|le|los|las|les)?)": {
        "base": "comed",
        "lemma": "comer",
        "upos": "VERB"
    },
    r"(?:C|c)ométe((?:me|te|se|nos|os)?)((?:lo|la|le|los|las|les)?)": {
        "base": "comete",
        "lemma": "cometer",
        "upos": "VERB"
    },
    r"(?:C|c)omet[eé]d((?:me|te|se|nos|os)?)((?:lo|le|la|los|les|las)?)": {
        "base": "cometed",
        "lemma": "cometer",
        "upos": "VERB"
    },
    r"(?:v|V)íste((?:me|te|se|nos|os)?)((?:lo|la|los|las)?)": {
        "base": "viste",
        "lemma": "vestir",
        "upos": "VERB"
    },
    r"(?:a|A)cués(?:ta|te)((?:me|te|se|nos|os)?)((?:lo|la|le|los|las|les)?)": {
        "base": "acuesta",
        "lemma": "acostar",
        "upos": "VERB"
    },
    r"(?:a|A)costad((?:me|te|se|nos|os)?)((?:lo|la|le|los|las|les)?)": {
        "base": "acosta",
        "lemma": "acostar",
        "upos": "VERB"
    },
    r"(?:d|D)[íi]((?:me|te|se|nos|os)?)((?:lo|la|los|las)?)": {
        "base": "di",
        "lemma": "decir",
        "upos": "VERB"
    },
    r"(?:m|M)uéstra((?:me|te|se|nos|os)?)((?:lo|la|los|las)?)": {
        "base": "muestra",
        "lemma": "mostrar",
        "upos": "VERB"
    },
    r"(?:j|J)ugu[ée]mo((?:nos))((?:lo|la|le|los|las|les)?)": {
        "base": "juguemo",
        "lemma": "jugar",
        "upos": "VERB"
    },
    r"(?:j|J)uguémos((?:lo|la|le|los|las|les)?)": {
        "base": "juguemos",
        "lemma": "jugar",
        "upos": "VERB"
    },
    r"(?:j|J)uéga((?:me|te|se|nos)?)((?:lo|la|le|los|las|les)?)": {
        "base": "juega",
        "lemma": "jugar",
        "upos": "VERB"
    },
    r"(?:j|J)ugáz((?:me|te|se|nos)?)((?:lo|la|le|los|las|les)?)": {
        "base": "jugad",
        "lemma": "jugar",
        "upos": "VERB"
    },
    r"(?:j|J)ugáos((?:lo|la|los|las))": {
        "base": "jugad",
        "lemma": "jugar",
        "upos": "VERB"
    },
    # Añadir más verbos según necesidad
}

In [ ]:
with open('patterns3.json', 'r', encoding='utf-8') as f:
    patterns = json.load(f)
    
CLITICS = ["me", "te", "se", "nos", "os", "lo", "la", "le", "los", "las", "les"]

In [ ]:
def split_clitics(clitic_string: str, clitics_list: list = list()):
    clitics = []
    remaining = clitic_string

    for cl in clitics_list:
        if remaining.startswith(cl):
            clitics.append(cl)
            remaining = remaining[len(cl):]

    return clitics

In [ ]:
def preprocess_imperatives(text: str, patterns: list | None = list()):
    imperative_meta = []

    def make_replacer(pattern: str, rule: dict):
        regex = re.compile(pattern, flags=re.IGNORECASE)

        def replacer(match):
            groups = match.groups() # Analiza el patrón para buscar que clíticos coinciden
            # Mete los clíticos que coincidan en una lista como elementos separados
            clitic_string = "".join(g for g in groups if g) if groups else ""
            clitics = split_clitics(clitic_string, CLITICS)

            # Si no hay clíticos, no tocar la palabra original
            if not clitics:
                return match.group(0)
            

            replacement = " ".join([rule["base"]] + clitics)

            imperative_meta.append({
                "base": rule["base"],
                "lemma": rule["lemma"],
                "upos": rule["upos"]
            })

            # Mantener mayúscula inicial
            if match.group(0)[0].isupper():
                replacement = replacement.capitalize()

            return replacement

        return regex, replacer

    for pattern, rule in IMPERATIVE_MAP.items():
        regex, replacer = make_replacer(pattern, rule)
        text = regex.sub(replacer, text)

    return text, imperative_meta

In [ ]:
def pos_tag(corpus: list[str] = []):
    
    nlp = stanza.Pipeline(
    lang="es",
    processors="tokenize,mwt,pos,lemma,depparse",
    tokenize_pretokenized=False,
    use_gpu=False,
    verbose=False
    )
    
    return [nlp(text.lower()) for text in corpus]
    # docs = []
    # for text in corpus: docs.append(nlp(text.lower()))
    # return docs 

In [ ]:
def print_tags(docs: list = []):
    for doc in docs:
        for token in doc:
            print(token.text, token.pos_)
        print()    

In [ ]:
def preprocesar(data: list[dict] | None = None):
    # Coger solo los textos
    texts = [d['text'] for d in data]
    # Separar clíticos
    preprocessed, imperative_meta = preprocess_imperatives(texts, patterns) 
    # Etiquetar textos
    doc = pos_tag(preprocessed)
    # Mostrar las etiquetas
    print_tags(doc[:5])
    
    return doc, imperative_meta

# PASAR TEXTO A FORMATO CONLLU

En este notebook se pasará de texto bruto en español a texto en formato conllu. Para ello se emplearán el tokenizador de stanza y funciones auxiliares que pretenden solucionar una de las carencias de stanza que son las formas imperativas de los verbos en español cuando estas llevan pronombres clíticos.

El formato conllu se construye de la siguiente forma:

1- Se incluyen líneas comentadas con el documento al que pertenece la frase, un identificador que nos diga que frase es dentro del documento y la frase original antes del tokenizado

2- Una linea para cada palabra que constará de 10 columnas:
 - **ID**: Índice del token. Su posición dentro de la oración.
 - **FORM**: Forma de la palabra. Palabra que aparece tal cual en el texto original.
 - **LEMMA**: Forma base de la palabra.
 - **UPOS**: Categoría gramatical universal de la palabra.
 - **XPOS**: Etiqueta gramatical específica del idioma.
 - **FEATS**: Lista de rasgos morfológicos (género, número, tiempo, persona).
 - **HEAD**: El ID de la palabra de la que depende.
 - **DEPREL**: El tipo de relación con el HEAD.
 - **DEPS**: Grafo de dependencias mejorado.
 - **MISC**: Cualquier otra información.

#### IMPORTS
- re: Librería para expresiones regulares
- stanza: Librería para tokenizar texto en español
- List y Dict: Librerías que representan listas y diccionarios

#### INICIALIZAR STANZA (ejecutarlo solo una vez)
stanza.pipeline: El pipeline coge texto o documentos de texto y lanza procesadores para devolver el texto analalizdo.
lang = Idioma del texto a analizar.
processors = Que quieres procesar del texto.
- tokenize: Divide el texto en oraciones y palabras (Tokens).
- mwt (Multi Word Tokenize): Expande los tokens de palabras compuestas en varias palabras cuando el tokenizador los predice. Un ejemplo de esto sería en español las palabras como al -> a + el y del -> de + el y en inglés las palabras como isn't -> is + not y aren't -> are + not.
- pos: Etiqueta los tokens con su categoría gramatical
- lemma: Genera los lemas de palabras para todas las palabras del documento.
- depparse: Proporciona un análisis de dependencia sintáctica preciso.
tokenize_pretokenized:
- False: la entrada es texto bruto. 
- True: la entrada es texto dividido en oraciones o lista de palabras 
use_gpu: 
- False: Indica que el procesamiento se hará usando la CPU de tu ordenador. 
- True: Si tienes una tarjeta gráfica configurada puedes activarlo para aumentar la velocidad.

In [ ]:
stanza.download("es")

nlp = stanza.Pipeline(
    lang="es",
    processors="tokenize,mwt,pos,lemma,depparse",
    tokenize_pretokenized=False,
    use_gpu=False,
    verbose=False
)

2026-01-06 13:56:04 INFO: Downloaded file to C:\Users\ivire\stanza_resources\resources.json
2026-01-06 13:56:04 INFO: Downloading default packages for language: es (Spanish) ...
2026-01-06 13:56:08 INFO: File exists: C:\Users\ivire\stanza_resources\es\default.zip
2026-01-06 13:56:14 INFO: Finished downloading models and saved to C:\Users\ivire\stanza_resources
2026-01-06 13:56:14 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2026-01-06 13:56:15 INFO: Downloaded file to C:\Users\ivire\stanza_resources\resources.json
2026-01-06 13:56:16 INFO: Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

2026-01-06 13:56:

#### LISTA DE IMPERATIVOS CON CLÍTICOS

Lista de excepciones que el tokenizador de stanza no sabe tokenizar, típicamente son las formas imperativas de los verbos en español que presentan clíticos.

Los verbos en la lista se reconocen mediante expresiones regulares de la base de la palabra y todas las combinaciones de clíticos que nos podemos encontrar en ella. También se especifica de que verbo viene ya que la base puede coincidir con alguna forma verbal de algún otro verbo como sucede con "díselo" cuya base es di que coincide con el pasado del verbo "dar". Por eso se le indica el lema al que pertenece y el upos ya que aunque todos sean verbos, el tokenizador no lo reconoce e intenta adivinarlo dándole alguna upos aleatoria.

DOCUMENTO STANZA → FORMATO CoNLL-U

**stanza_doc_to_conllu**

Entrada: 
- Texto tokenizado por stanza
- Lista de palabras a las que hay que cambiarles algún campo

Salida: Texto en formato conllu

Recorreras cada oración analizada por stanza y escribirás los encabezados que identifican cada frase. Después se cambiarán de forma manual los lemmas y upos de las palabras que coincidieron con algún patrón de la lista de excepciones para que no se pierda el significado.

Una vez hecho esto, el código recolectará los atributos del análisis que stanza hizo previamente y los dispondrá en el formato conllu final

In [8]:
def stanza_doc_to_conllu(doc, imperative_meta) -> str:
    lines = []
    sent_id = 1

    for sent in doc.sentences:
        # Identificadores de cada frase
        lines.append(f"# sent_id = {sent_id}")
        lines.append(f"# text = {sent.text}")

        for word in sent.words:
            lemma = word.lemma
            upos = word.upos

            # Corrección manual del lema si viene de whitelist
            for meta in imperative_meta:
                if word.text.lower() == meta["base"]:
                    lemma = meta["lemma"]
                    upos = meta["upos"]

            # Extracción de los atributos de las palabras
            feats = word.feats if word.feats else "_"

            misc = []
            if word.start_char is not None and word.end_char is not None:
                misc.append(f"CharOffset={word.start_char}:{word.end_char}")
            misc = "|".join(misc) if misc else "_"

            # Construcción del conllu final
            lines.append("\t".join([
                str(word.id),        # ID
                word.text,           # FORM
                lemma or "_",         # LEMMA (corregido)
                upos or "_",          # UPOS
                word.xpos or "_",     # XPOS
                feats,                # FEATS
                str(word.head),       # HEAD
                word.deprel or "_",   # DEPREL
                "_",                  # DEPS
                misc                  # MISC
            ]))

        lines.append("")
        sent_id += 1

    return "\n".join(lines)

FUNCIÓN PRUNCIPAL: TEXTO(S) → CoNLL-U

**texts_to_conllu**

Esta celda es la función principal que irá llamando a las funciones de las celdas anteriores.

 1º- Sustituye los verbos con clíticos por palabras separadas. Ej: dímelo -> di + me + lo

 2º- Tokenizarás cada palabra con stanza.pipeline

 3º- Construye la salida del preprocesado en un texto formato conllu

In [ ]:
def texts_to_conllu(texts: List[str]) -> str:
    docs = []

    for i, text in enumerate(texts, start=1):
        # Separar clíticos y tokenizar las palabras de la frase (Parte del notebook de preprocesar)
        doc, imperative_meta = parse_text(text)

        # Dar formato conllu a los tokens de la frase
        conllu = stanza_doc_to_conllu(doc, imperative_meta)
        
        # Separación entre frases
        docs.append(f"# newdoc id = doc_{i}")
        docs.append(conllu)

    return "\n".join(docs)

EJEMPLO DE USO

**texto_a_frases**

Convierte texto bruto en una lista de frases. Separa el texto cuando encuentra algún signo de puntuación que indique el fin de frase.

Esto es necesario para llamar a la función texts_to_conllu ya que espera una lista

In [10]:
def texto_a_frases(texto: str) -> list[str]:
    if not texto or not texto.strip():
        return []

    # Normalizar espacios
    texto = re.sub(r'\s+', ' ', texto.strip())

    # Separar por fin de frase
    frases = re.split(r'(?<=[.!?¿¡])\s+', texto)

    # Limpiar frases vacías
    return [f.strip() for f in frases if f.strip()]

### MAIN del programa

In [14]:
texto = "Cómeselo. Comételos. Comédmelo. Cometedlo"
corpus = texto_a_frases(texto)

conllu_output = texts_to_conllu(corpus)
print(conllu_output)

# newdoc id = doc_1
# sent_id = 1
# text = Come se lo.
1	Come	comer	VERB	vmip3s0	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	0	root	_	CharOffset=0:4
2	se	él	PRON	_	Case=Acc|Person=3|PrepCase=Npr|PronType=Prs|Reflex=Yes	1	expl:pv	_	CharOffset=5:7
3	lo	él	PRON	_	Case=Acc|Gender=Masc|Number=Sing|Person=3|PrepCase=Npr|PronType=Prs	1	obj	_	CharOffset=8:10
4	.	.	PUNCT	fp	PunctType=Peri	1	punct	_	CharOffset=10:11

# newdoc id = doc_2
# sent_id = 1
# text = Comete los.
1	Comete	cometer	VERB	vmip3s0	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	0	root	_	CharOffset=0:6
2	los	él	PRON	_	Case=Acc|Gender=Masc|Number=Plur|Person=3|PrepCase=Npr|PronType=Prs	1	obj	_	CharOffset=7:10
3	.	.	PUNCT	fp	PunctType=Peri	1	punct	_	CharOffset=10:11

# newdoc id = doc_3
# sent_id = 1
# text = Comed me lo.
1	Comed	comer	VERB	vmip1s0	VerbForm=Fin	0	root	_	CharOffset=0:5
2	me	yo	PRON	_	Case=Dat|Number=Sing|Person=1|PrepCase=Npr|PronType=Prs	1	obl:arg	_	CharOffset=6:8
3	lo	él	PRON	_	Case=Acc|Gender=M

Guarda la salida del text_to_conllu en un archivo (entrada.conllu) que se usará como entrada del modelo a entrenar

In [12]:
file_name = "entrada.conllu"

with open(file_name, "w", encoding="utf-8") as f:
    f.write(conllu_output)

print(f"Archivo guardado como {file_name}")

Archivo guardado como entrada.conllu
